In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 39.1 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
     |████████████████████████████████| 6.6 MB 57.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 31.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 18.4 MB/s 
     |████████████████████████████████| 212 kB 45.1 MB/s 
     |████████████████████████████████| 115 kB 55.8 MB/s 
     |████████████████████████████████| 127 kB 54.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [3]:
%cd /content/drive/MyDrive/Thac Si/Thesis/BaViBARTModel

/content/drive/MyDrive/Thac Si/Thesis/BaViBARTModel


In [ ]:
!python custom_dataset/create_custom_tokenizer.py

100% 1987/1987 [00:00<00:00, 42588.19it/s]
100% 9275/9275 [00:00<00:00, 34099.73it/s]
100% 1988/1988 [00:00<00:00, 25212.27it/s]
3
['Adrĭng', 'Sơ', 'Thông', 'tin', 'weng', 'Trŭn', 'thông', ',', 'rim', 'đơn', 'vĭ', 'liên', 'kuan', 'chuân', "'", 'bi', 'rim', 'âÊn', 'phâm', 'pơtho', 'khan', ',', 'pơlĕi', 'rim', 'măng', 'hŏp', "'", 'băo']
<s> Adrĭng Sơ Thông tin weng Trŭn thông,rim đơn vĭ liên kuan chuân 'birim âÊn phâm pơtho khan, pơlĕi rim măng hŏp'băo </s>


In [4]:
!nvidia-smi

Mon Aug 22 15:15:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from abc import ABC
import torch
from transformers import EncoderDecoderModel

from transformers.models.mbart import MBartForConditionalGeneration, MBartConfig
from transformers.modeling_outputs import Seq2SeqLMOutput
from transformers.models.mbart.modeling_mbart import (
    MBART_INPUTS_DOCSTRING,
    _CONFIG_FOR_DOC,
    MBART_GENERATION_EXAMPLE,
    shift_tokens_right
)
from transformers.file_utils import (
    add_code_sample_docstrings,
    add_end_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    replace_return_docstrings,
)
from torch.nn import CrossEntropyLoss
import numpy as np

bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")


class CustomMbartModel(MBartForConditionalGeneration, ABC):
    def __init__(self, config: MBartConfig):
        super(CustomMbartModel, self).__init__(config)
        self.word_dropout_ratio = None
        self.word_replacement_ratio = None
        self.pad_id = config.pad_token_id
        self.bos_id = config.bos_token_id
        self.eos_id = config.eos_token_id

    def set_augment_config(self, word_dropout_ratio: float = 0, word_replacement_ratio: float = 0):
        self.word_dropout_ratio = word_dropout_ratio
        self.word_replacement_ratio = word_replacement_ratio

    def word_dropout(self, batch_):
        if self.word_dropout_ratio == 0:
            return batch_
        batch = torch.clone(batch_)
        for batch_idx in range(len(batch)):
            for token_idx in range(len(batch[batch_idx])):
                if np.random.randn(1)[0] < self.word_dropout_ratio:
                    batch[batch_idx][token_idx] *= 0
        return batch

    def word_replacement(self, batch_ids_):
        if self.word_replacement_ratio == 0:
            return batch_ids_
        batch_ids = torch.clone(batch_ids_)
        embedding_size = len(self.model.shared.weight)
        for batch_idx in range(len(batch_ids)):
            for token_idx in range(len(batch_ids[batch_idx])):
                if token_idx == self.pad_id:
                    break
                if np.random.randn(1)[0] < self.word_replacement_ratio:
                    batch_ids[batch_idx][token_idx] = np.random.randint(0, embedding_size)
        return batch_ids

    @add_start_docstrings_to_model_forward(MBART_INPUTS_DOCSTRING)
    @replace_return_docstrings(output_type=Seq2SeqLMOutput, config_class=_CONFIG_FOR_DOC)
    @add_end_docstrings(MBART_GENERATION_EXAMPLE)
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        decoder_input_ids=None,
        decoder_attention_mask=None,
        head_mask=None,
        decoder_head_mask=None,
        cross_attn_head_mask=None,
        encoder_outputs=None,
        past_key_values=None,
        inputs_embeds=None,
        decoder_inputs_embeds=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Labels for computing the masked language modeling loss. Indices should either be in ``[0, ...,
            config.vocab_size]`` or -100 (see ``input_ids`` docstring). Tokens with indices set to ``-100`` are ignored
            (masked), the loss is only computed for the tokens with labels in ``[0, ..., config.vocab_size]``.

        Returns:

        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if labels is not None:
            input_ids = self.word_replacement(input_ids)
            decoder_input_ids = self.word_replacement(labels)
            decoder_input_ids = shift_tokens_right(decoder_input_ids, self.config.pad_token_id)

        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            encoder_outputs=encoder_outputs,
            decoder_attention_mask=decoder_attention_mask,
            head_mask=head_mask,
            decoder_head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            decoder_inputs_embeds=decoder_inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        lm_logits = self.lm_head(outputs[0]) + self.final_logits_bias

        masked_lm_loss = None
        if labels is not None:
            loss_fct = CrossEntropyLoss()
            masked_lm_loss = loss_fct(lm_logits.view(-1, self.config.vocab_size), labels.view(-1))

        if not return_dict:
            output = (lm_logits,) + outputs[1:]
            return ((masked_lm_loss,) + output) if masked_lm_loss is not None else output

        return Seq2SeqLMOutput(
            loss=masked_lm_loss,
            logits=lm_logits,
            past_key_values=outputs.past_key_values,
            decoder_hidden_states=outputs.decoder_hidden_states,
            decoder_attentions=outputs.decoder_attentions,
            cross_attentions=outputs.cross_attentions,
            encoder_last_hidden_state=outputs.encoder_last_hidden_state,
            encoder_hidden_states=outputs.encoder_hidden_states,
            encoder_attentions=outputs.encoder_attentions,
        )


In [1]:
import transformers
from datasets import load_metric
from transformers import AutoTokenizer
from transformers.models.mbart import MBartForConditionalGeneration
from model.custom_mbart_model import CustomMbartModel
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np

from custom_dataset.vi_ba_dataset import ViBaDataset


def get_metric(metric_, tokenizer_):
    def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]
        return preds, labels

    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer_.batch_decode(preds, skip_special_tokens=True)

        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer_.pad_token_id)
        decoded_labels = tokenizer_.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        result = metric_.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"bleu": result["score"]}

        prediction_lens = [np.count_nonzero(pred != tokenizer_.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        return result

    return compute_metrics


def main():
    WORD_DROPOUT_RATIO = 0.15
    WORD_REPLACEMENT_RATIO = 0.15
    model_checkpoint = "pretrained/bartpho_syllable"
    metric = load_metric("sacrebleu")
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    model = CustomMbartModel.from_pretrained(model_checkpoint)
    model.resize_token_embeddings(len(tokenizer.get_vocab()))
    model.set_augment_config(word_dropout_ratio=WORD_DROPOUT_RATIO,
                             word_replacement_ratio=WORD_REPLACEMENT_RATIO)
    compute_metric_func = get_metric(metric, tokenizer)

    train_dataset, valid_dataset, test_dataset = ViBaDataset.get_datasets(data_folder="data/all",
                                                                          tokenizer_path=model_checkpoint)

    batch_size = 4

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    trainer_args = Seq2SeqTrainingArguments(
        "/content/checkpoint/viba_bart-finetuned",
        metric_for_best_model="bleu",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=4,
        predict_with_generate=True,
        fp16=False,
        push_to_hub=False,
        logging_steps=100,
        logging_first_step=True,
        logging_dir="logging/viba_bart-finetuned",
        eval_steps=100
    )

    trainer = Seq2SeqTrainer(
        model,
        trainer_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metric_func
    )
    print(trainer.evaluate(eval_dataset=valid_dataset, num_beams=5, max_length=512))

    trainer.train()

    print(trainer.evaluate(test_dataset, num_beams=5, max_length=512))
    trainer.save_model("checkpoint/best_4")


e:\General_Subjects\artificial_intelligence\EAAI\231-IntProject\bart-viba\khangEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import string
import transformers
from datasets import load_metric
from transformers import AutoTokenizer
from transformers.models.mbart import MBartForConditionalGeneration
from model.custom_mbart_model import CustomMbartModel
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np

from custom_dataset.vi_ba_dataset import ViBaDataset


def get_metric(metric_, tokenizer_):
    def norm_text(line):
        for n in string.digits:
            line = line.replace(n, f" {n} ")
        for p in string.punctuation + ",.\\/:;–…":
            line = line.replace(p, f" {p} ")
        while "  " in line:
            line = line.replace("  ", " ")
        line = line.strip()
        return line

    def postprocess_text(preds, labels):
        preds = [norm_text(pred) for pred in preds]
        labels = [[norm_text(label)] for label in labels]
        return preds, labels

    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer_.batch_decode(preds, skip_special_tokens=True)

        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer_.pad_token_id)
        decoded_labels = tokenizer_.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        result = metric_.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"bleu": result["score"]}

        prediction_lens = [np.count_nonzero(pred != tokenizer_.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        return result

    return compute_metrics


def main():
    WORD_DROPOUT_RATIO = 0.15
    WORD_REPLACEMENT_RATIO = 0.15
    model_checkpoint = "/content/drive/MyDrive/Thac Si/Thesis/BaViBARTModel/pretrained/bartpho_syllable"
    metric = load_metric("sacrebleu")
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    a = tokenizer.encode("Kjung tơgŭm tơjung pơm kơ sơ ha tâng, tơjung pơm cham pơlĕi anao weng rim tơdrong pôk xuôn tơdrong yăl arih kon tơring lơ̆m tơring tinh, Chu tich UBND tinh pơtho khan pơgơl rim sơ, 'ban, nganh, tô chưk chinh tri sa hôi, đoan thê tinh")
    print(a)
    model = CustomMbartModel.from_pretrained(model_checkpoint)
    model.set_augment_config(word_dropout_ratio=WORD_DROPOUT_RATIO,
                             word_replacement_ratio=WORD_REPLACEMENT_RATIO)
    compute_metric_func = get_metric(metric, tokenizer)

    train_dataset, valid_dataset, test_dataset = ViBaDataset.get_datasets(data_folder="data/all",
                                                                          tokenizer_path=model_checkpoint)

    batch_size = 16

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    trainer_args = Seq2SeqTrainingArguments(
        "/content/checkpoint/viba_bart-finetuned",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=4,
        predict_with_generate=True,
        fp16=False,
        push_to_hub=False,
        logging_steps=100,
        logging_first_step=True,
        logging_dir="logging/viba_bart-finetuned",
        eval_steps=100,
    )

    trainer = Seq2SeqTrainer(
        model,
        trainer_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metric_func
    )

    # trainer.train()

    output = trainer.evaluate(test_dataset, num_beams=5, max_length=1024)
    print(output)

main()


Streaming output truncated to the last 5000 lines.
Adding akuk to the vocabulary
Adding Truyên to the vocabulary
Adding kenh to the vocabulary
Adding tơhlăk to the vocabulary
Adding akoih to the vocabulary
Adding tác to the vocabulary
Adding chê̆ to the vocabulary
Adding toh to the vocabulary
Adding triên to the vocabulary
Adding ruô̆k to the vocabulary
Adding yuiêt to the vocabulary
Adding bư̆a to the vocabulary
Adding piêng to the vocabulary
Adding kơmăng to the vocabulary
Adding Hổ to the vocabulary
Adding luôi to the vocabulary
Adding jăp to the vocabulary
Adding QLCL to the vocabulary
Adding kô to the vocabulary
Adding gôp to the vocabulary
Adding bơgal to the vocabulary
Adding Thăi to the vocabulary
Adding tơmăn to the vocabulary
Adding Vui to the vocabulary
Adding Duch to the vocabulary
Adding Đuôĭ to the vocabulary
Adding sơhlung to the vocabulary
Adding Xã to the vocabulary
Adding pơđăm to the vocabulary
Adding Oêng to the vocabulary
Adding tơl to the vocabulary
Adding h

[0, 44182, 41863, 40246, 41836, 44160, 42645, 1728, 40092, 23, 4, 40246, 41836, 950, 35, 44009, 40168, 44124, 12503, 40783, 40265, 43591, 40783, 44375, 42071, 26248, 41224, 40332, 41224, 42379, 23, 4, 45391, 44977, 42282, 42379, 44110, 3706, 43622, 12503, 42645, 23, 4, 524, 3749, 4, 41847, 23, 4, 902, 41383, 43766, 1612, 2054, 43241, 23, 4, 41746, 40749, 42379, 2]


Streaming output truncated to the last 5000 lines.
Adding Aân to the vocabulary
Adding hê̆ to the vocabulary
Adding đơ̆i to the vocabulary
Adding Cao to the vocabulary
Adding yŏk… to the vocabulary
Adding kơlih to the vocabulary
Adding tơhgngam to the vocabulary
Adding hlă to the vocabulary
Adding pơlêch to the vocabulary
Adding vĕi to the vocabulary
Adding PTNT to the vocabulary
Adding tông to the vocabulary
Adding tơblah to the vocabulary
Adding sơđah to the vocabulary
Adding nôi to the vocabulary
Adding pơlôm to the vocabulary
Adding iêl… to the vocabulary
Adding tũk to the vocabulary
Adding Chơđah to the vocabulary
Adding oĕng to the vocabulary
Adding Lôk to the vocabulary
Adding nhuân… to the vocabulary
Adding YôlI to the vocabulary
Adding giay to the vocabulary
Adding thuôi to the vocabulary
Adding chơhlŭng to the vocabulary
Adding chdruh to the vocabulary
Adding kơtoh to the vocabulary
Adding gêr… to the vocabulary
Adding sơhmĕch to the vocabulary
Adding plăi to the vocabular

NameError: ignored

In [ ]:
from transformers import AutoTokenizer
from model.custom_mbart_model import CustomMbartModel
from datasets import load_metric
from custom_dataset.vi_ba_dataset import ViBaDataset

WORD_DROPOUT_RATIO = 0.15
WORD_REPLACEMENT_RATIO = 0.15
model_checkpoint = "/content/drive/MyDrive/Thac Si/Thesis/BaViBARTModel/checkpoint/best"
metric = load_metric("sacrebleu")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = CustomMbartModel.from_pretrained(model_checkpoint)
# compute_metric_func = get_metric(metric, tokenizer)

_, _, test_dataset = ViBaDataset.get_datasets(data_folder="data/all", tokenizer_path=model_checkpoint)

KeyboardInterrupt: ignored

In [ ]:
import torch 
from tqdm import tqdm 

data = []
for i, item in enumerate(tqdm(test_dataset)):
    # print(item)
    input_ids = item["input_ids"]
    labels = item["labels"]
    input_ids = torch.tensor([input_ids])
    outputs = model.generate(input_ids=input_ids, num_beams=5, max_length=512, num_return_sequences=1)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if i % 10 == 0:
        print(output)
    data.append(output)
    with open("/content/drive/MyDrive/Thac Si/Thesis/BaViBARTModel/data/result.ba", "w") as f:
        f.write("\n".join(data))

  0%|          | 1/1987 [00:06<3:29:03,  6.32s/it]

Adrĭng Sơ Thông tin weng Trŭn thông, rim đơn vĭ liên kuan chuân'bi rim âÊn phâm pơtho khan, pơlĕi rim măng hŏp'băo


  1%|          | 11/1987 [01:16<2:46:31,  5.06s/it]

Dơnŏ phô Quy Nhơn, thi sa An Nhơn, hŭn Tây Sơn, hŭn Vinh Thanh, hŭn Tây Sơn


  1%|          | 21/1987 [02:36<4:00:22,  7.34s/it]

...

 99%|█████████▉| 1971/1987 [3:38:50<02:17,  8.57s/it]

( AÊn lêch pơtêp adrĭng Kŭ̆t đinh pêng minh puôn tơpơ̆h / QĐ - BTP'năr'băl jĭt tơhngam khĕi minh jĭt'băl sơnăm'băl ơbơ̆u khôn g hriêng minh jĭt tơhngam đơ̆ng'Bô trương'Bô Tư phap )


100%|█████████▉| 1981/1987 [3:39:46<00:32,  5.37s/it]

chă


100%|██████████| 1987/1987 [3:40:12<00:00,  6.65s/it]


In [ ]:
!cp -r /content/checkpoint/viba_bart-finetuned/checkpoint-6000 /content/checkpoint-6000

In [ ]:
!zip -r /content/checkpoint6000.zip /content/checkpoint-6000

  adding: content/checkpoint-6000/ (stored 0%)
  adding: content/checkpoint-6000/special_tokens_map.json (deflated 50%)
  adding: content/checkpoint-6000/config.json (deflated 57%)
  adding: content/checkpoint-6000/sentencepiece.bpe.model (deflated 49%)
  adding: content/checkpoint-6000/rng_state.pth (deflated 27%)
  adding: content/checkpoint-6000/pytorch_model.bin (deflated 7%)
  adding: content/checkpoint-6000/tokenizer_config.json (deflated 50%)
  adding: content/checkpoint-6000/trainer_state.json (deflated 80%)
  adding: content/checkpoint-6000/scheduler.pt (deflated 49%)
  adding: content/checkpoint-6000/training_args.bin (deflated 49%)
  adding: content/checkpoint-6000/.ipynb_checkpoints/ (stored 0%)
  adding: content/checkpoint-6000/dict.txt (deflated 55%)


In [ ]:
import torch
from transformers import AutoTokenizer
from model.custom_mbart_model import CustomMbartModel

model_checkpoint = "/content/drive/MyDrive/Thac Si/Thesis/BaViBARTModel/checkpoint/best"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = CustomMbartModel.from_pretrained(model_checkpoint)

def translate(text):
    input_ids = tokenizer(text).input_ids
    input_ids = torch.tensor([input_ids])
    outputs = model.generate(input_ids=input_ids, num_beams=5, max_length=1024, num_return_sequences=1)
    print(outputs)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)[0]
    return output


In [ ]:
import string 

def norm_text(line):
    for n in string.digits:
        line = line.replace(n, f" {n} ")
    for p in string.punctuation + ",.\\/:;–…":
        line = line.replace(p, f" {p} ")
    while "  " in line:
        line = line.replace("  ", " ")
    line = line.strip()
    return line

def translate(text):
    text = norm_text(text)
    input_ids = tokenizer(text).input_ids
    input_ids = torch.tensor([input_ids])
    outputs = model.generate(input_ids=input_ids, num_beams=5, max_length=1024, num_return_sequences=1)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output

print(translate("Tôi là sinh viên đại học"))
print(translate("Tôi là sinh viên đại học bách khoa"))
print(translate("hôm nay"))
print(translate("tôi đi học"))
print(translate("anh khỏe không "))


In [ ]:
print(translate("anh khỏe không ạ?"))

Năm hăm?


In [ ]:
from transformers import MBartForConditionalGeneration

bartpho = MBartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Thac Si/Thesis/BaViBARTModel/checkpoint/best")
TXT = "Inh la sinh viên đai hŏk <mask> Ho Chi Minh."
input_ids = tokenizer([TXT], return_tensors="pt")["input_ids"]
logits = bartpho(input_ids).logits
masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
probs = logits[0, masked_index].softmax(dim=0)
values, predictions = probs.topk(5)
print(tokenizer.decode(predictions).split())

loading configuration file /content/drive/MyDrive/Thac Si/Thesis/BaViBARTModel/checkpoint/best/config.json
Model config MBartConfig {
  "_name_or_path": "pretrained/bartpho_syllable",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "CustomMbartModel"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_position_embeddings": 1024,
  "model_type": "mbart",
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "scale_embedding": false,
  "tokenizer_class": "BartphoTokenizer",
  "torch_dtype": "float3

['Ho', 'ho', 'tơ', 'Thanh']


In [ ]:
!cp /content/checkpoint6000.zip  /content/drive/MyDrive/BaViBARTModel/checkpoint/viba_bart-finetuned

In [ ]:
!cp /content/drive/MyDrive/BaViBARTModel/checkpoint/viba_bart-finetuned/checkpoint6000.zip /content/

In [ ]:
!rm -r ckpt

In [ ]:
!unzip /content/checkpoint6000.zip -d /content/ckpt 

Archive:  /content/checkpoint6000.zip
   creating: /content/ckpt/content/checkpoint-6000/
  inflating: /content/ckpt/content/checkpoint-6000/special_tokens_map.json  
  inflating: /content/ckpt/content/checkpoint-6000/config.json  
  inflating: /content/ckpt/content/checkpoint-6000/sentencepiece.bpe.model  
  inflating: /content/ckpt/content/checkpoint-6000/rng_state.pth  
  inflating: /content/ckpt/content/checkpoint-6000/pytorch_model.bin  
  inflating: /content/ckpt/content/checkpoint-6000/tokenizer_config.json  
  inflating: /content/ckpt/content/checkpoint-6000/trainer_state.json  
  inflating: /content/ckpt/content/checkpoint-6000/scheduler.pt  
  inflating: /content/ckpt/content/checkpoint-6000/training_args.bin  
   creating: /content/ckpt/content/checkpoint-6000/.ipynb_checkpoints/
  inflating: /content/ckpt/content/checkpoint-6000/dict.txt  


In [ ]:
import string
from nltk.translate.bleu_score import corpus_bleu

data_path = "/content/drive/MyDrive/Thac Si/Thesis/BaViBARTModel/data/result.ba"
ref_path = "/content/drive/MyDrive/Thac Si/Thesis/BaViBARTModel/data/all/test.ba"

def norm_text(line):
    for n in string.digits:
        line = line.replace(n, f" {n} ")
    for p in string.punctuation + ",.\\/:;–…":
        line = line.replace(p, f" {p} ")
    while "  " in line:
        line = line.replace("  ", " ")
    line = line.strip()
    return line

def read(path):
    data = open(path, "r").readlines()
    data = [norm_text(item.replace("\n", "")) for item in data]
    return data

pred = read(data_path)
ref = [[item] for item in read(ref_path)]

score = corpus_bleu(ref, pred)
print(score)


0.8143898899780662


In [ ]:
print(len(pred), len(ref))

1176 1176
